<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width="35%" align="right">


# yoctoGPT — Minimal GPT from Scratch

## _Getting Started — Character-level Training_

**&copy; Dr. Yves J. Hilpisch**<br>AI-Powered by OpenAI & Gemini.

## How to Use This Notebook

- **Goal**: Train a tiny GPT model on a character-level corpus in minutes.
- **Hardware**: Designed for Google Colab L4 (free tier) or better.
- **Approach**: We use `yoctoGPT`, a minimal PyTorch implementation focused on clarity.

### Roadmap

1. **Setup**: Install dependencies and clone the `yoctoGPT` repository.
2. **Data**: Prepare a character-level dataset from a philosophical text.
3. **Training**: Execute a short training run with a small model parametrization.
4. **Sampling**: Generate text from the trained checkpoint to see the model in action.

### Setup: Environment and Dependencies

We first check the GPU availability and install the necessary requirements.

In [ ]:
#@title Check GPU and Install Dependencies
!nvidia-smi
!pip -q install tokenizers tqdm textstat

import os
import pathlib
import subprocess

# Clone the repository if it doesn't exist
repo_root = pathlib.Path("/content/yoctoGPT")
if not repo_root.exists():
    print("Cloning yoctoGPT repository...")
    subprocess.run([
        "git", "clone",
        "https://github.com/yhilpisch/yoctoGPT.git",
        str(repo_root)
    ])
os.chdir(repo_root)
print(f"Current working directory: {os.getcwd()}")

### Data Preparation

We use a character-level encoding where each character (letter, digit, punctuation) is assigned a unique integer ID. This is the simplest way to start building a language model.

In [ ]:
#@title Prepare Character-level Data
!python -m scripts.prepare_char_data \
    --text_path data/philosophy.txt \
    --out_dir data/char_start \
    --sanitize_chars ascii \
    --no_punctuation \
    --keep_period \
    --collapse_whitespace \
    --lowercase

### Model Training

We initialize a small GPT model. Smaller models train faster and are perfect for learning the core mechanics without waiting for hours.

**Configuration:**
- `n_layer`: 2 (number of transformer blocks)
- `n_head`: 4 (number of attention heads)
- `n_embd`: 128 (embedding dimension)
- `block_size`: 128 (context window length)

In [ ]:
#@title Train Tiny yoctoGPT
# This automatically saves checkpoints (best.pt, latest.pt) to --ckpt_dir
!python -m yoctoGPT.train \
    --mode char \
    --data_dir data/char_start \
    --ckpt_dir checkpoints/char_start \
    --n_layer 2 \
    --n_head 4 \
    --n_embd 128 \
    --block_size 128 \
    --batch_size 32 \
    --max_iters 500 \
    --lr 1e-3 \
    --eval_interval 100

# Verify checkpoint creation
!ls -lh checkpoints/char_start/latest.pt

### Resume Training

You can continue training from a saved checkpoint to further improve the model's performance. This uses the `latest.pt` file saved in your checkpoint directory.

In [ ]:
#@title Resume training from latest.pt
latest_path = "checkpoints/char_start/latest.pt"
if os.path.exists(latest_path):
    !python -m yoctoGPT.train \
        --mode char \
        --data_dir data/char_start \
        --ckpt_dir checkpoints/char_start \
        --resume {latest_path} \
        --n_layer 2 \
        --n_head 4 \
        --n_embd 128 \
        --block_size 128 \
        --max_iters 2500
else:
    print(f"No checkpoint found at {latest_path}. Run the initial training first.")

### Text Generation (Sampling)

Once trained, we can prompt the model to generate new text based on what it learned from the corpus.

In [ ]:
#@title Sample from the Model
prompt = "the meaning of life is"
output = !python -m yoctoGPT.sampler \
    --mode char \
    --ckpt checkpoints/char_start/latest.pt \
    --vocab_path data/char_start/vocab.json \
    --prompt "{prompt}" \
    --max_new_tokens 200

generated_text = "\n".join(output)
print(generated_text)

### Extended Training on the Same Corpus

The first run above is intentionally tiny so you can see end-to-end behavior in a few minutes.
This block keeps the same small character corpus (`philosophy.txt`) but trains a larger transformer from scratch with a more realistic configuration.

Use the same sampling and readability cells to compare outputs and scores from the tiny run vs this deeper run.


In [ ]:
#@title Train deeper yoctoGPT on the same corpus
!python -m yoctoGPT.train \
    --mode char \
    --data_dir data/char_start \
    --ckpt_dir checkpoints/char_deeper \
    --n_layer 6 \
    --n_head 8 \
    --n_embd 256 \
    --block_size 256 \
    --batch_size 32 \
    --max_iters 1500 \
    --lr 3e-4 \
    --eval_interval 500

In [ ]:
#@title Same idea, but better utilization of the L4 card
!python -m yoctoGPT.train \
    --mode char \
    --data_dir data/char_start \
    --ckpt_dir checkpoints/char_deeper_l4_tuned \
    --model_type gpt_fast \
    --n_layer 8 \
    --n_head 8 \
    --n_embd 512 \
    --block_size 384 \
    --batch_size 128 \
    --max_iters 1500 \
    --lr 1.2e-4 \
    --amp \
    --amp_dtype bf16 \
    --eval_interval 250 \
    --eval_iters 30 \
    --auto_microbatch \
    --cosine_lr \
    --warmup_iters 100

In [ ]:
#@title Resume from best checkpoint (lower LR, early stopping)
!python -m yoctoGPT.train \
    --mode char \
    --data_dir data/char_start \
    --ckpt_dir checkpoints/char_deeper_l4_tuned \
    --resume checkpoints/char_deeper_l4_tuned/best.pt \
    --model_type gpt_fast \
    --n_layer 8 \
    --n_head 8 \
    --n_embd 512 \
    --block_size 384 \
    --batch_size 128 \
    --max_iters 750 \
    --lr 2e-5 \
    --amp \
    --amp_dtype bf16 \
    --eval_interval 250 \
    --eval_iters 60 \
    --auto_microbatch \
    --cosine_lr \
    --warmup_iters 50 \
    --early_stopping_patience 3 \
    --early_stopping_min_delta 0.01

### Text Generation: Deeper Model

After training the larger transformer on the same corpus, you can sample from it
to see whether the outputs become less gibberish-like. Use the same prompt so
comparisons stay fair.


In [ ]:
#@title Sample from the deeper model
prompt = 'the meaning of life is'
output_deeper = !python -m yoctoGPT.sampler \
--mode char \
--ckpt checkpoints/char_deeper_l4_tuned/latest.pt \
--vocab_path data/char_start/vocab.json \
--prompt '{prompt}' \
--max_new_tokens 200

deeper_generated_text = '\n'.join(output_deeper)
print(deeper_generated_text)


### Readability Assessment

We evaluate the quality and complexity of the generated text using standard readability metrics. This helps us understand if the model is producing coherent, human-like structures or just random character sequences.

**Metric Meanings:**
- **Flesch Reading Ease**: Higher scores indicate text that is easier to read (100 is very easy, 0 is very difficult).
- **Flesch-Kincaid Grade**: Estimates the U.S. school grade level required to understand the text.
- **Dale-Chall Score**: Uses a list of 3,000 familiar words to assess complexity (lower is easier).
- **Text Standard**: A consensus metric that summarizes the estimated reading level (e.g., '8th Grade').

In [ ]:
#@title Analyze Generated Text Readability
from textstat import textstat
import json

def readability_scores(text: str) -> dict:
    return {
        "Flesch Reading Ease": textstat.flesch_reading_ease(text),
        "Flesch-Kincaid Grade": textstat.flesch_kincaid_grade(text),
        "Dale-Chall Score": textstat.dale_chall_readability_score(text),
        "Text Standard": textstat.text_standard(text, float_output=False),
    }

print("Readability Analysis (tiny model: checkpoints/char_start)")
print('-' * 40)
scores_tiny = readability_scores(generated_text)
for metric, value in scores_tiny.items():
    print(f'{metric:20}: {value}')

# If a deeper model sample exists, compare its scores as well
if 'deeper_generated_text' in globals():
    print("\nReadability Analysis (deeper model: checkpoints/char_deeper_l4_tuned)")
    print('-' * 40)
    scores_deeper = readability_scores(deeper_generated_text)
    for metric, value in scores_deeper.items():
        print(f'{metric:20}: {value}')


### Exercises

1. **Experiment with Context**: Increase the `block_size` to 256 and see how it affects memory usage (VRAM) and training speed.
2. **Deeper Model**: Change `n_layer` to 4 and observe the validation loss after 500 iterations. Does it improve?
3. **Custom Corpus**: Upload your own `.txt` file to the `data/` folder and retrain the model on your own text.

<img src="https://theaiengineer.dev/tae_logo_gw_flatter.png" width="35%" align="right">
